In [2]:
from datasets import load_dataset, Dataset

In [3]:
ot = load_dataset("open-thoughts/OpenThoughts-114k", split="train")

README.md:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

train-00000-of-00006.parquet:   0%|          | 0.00/250M [00:00<?, ?B/s]

train-00001-of-00006.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

train-00002-of-00006.parquet:   0%|          | 0.00/173M [00:00<?, ?B/s]

train-00003-of-00006.parquet:   0%|          | 0.00/174M [00:00<?, ?B/s]

train-00004-of-00006.parquet:   0%|          | 0.00/154M [00:00<?, ?B/s]

train-00005-of-00006.parquet:   0%|          | 0.00/152M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/113957 [00:00<?, ? examples/s]

In [6]:
deepscale = load_dataset("agentica-org/DeepScaleR-Preview-Dataset", split="train")

README.md:   0%|          | 0.00/1.78k [00:00<?, ?B/s]

deepscaler.json:   0%|          | 0.00/21.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40315 [00:00<?, ? examples/s]

In [47]:
import textwrap
from typing import List, Dict

def print_chatml(messages: List[Dict[str, str]], width: int = 80):
    """
    Minimal print function for ChatML conversations.
    
    Args:
        messages: List of ChatML message dictionaries with 'role' and 'content' keys
        width: Maximum width of output in characters
    """
    for message in messages:
        role = message.get("from", "unknown")
        content = message.get("value", "")
        
        # Print role header
        print(f"\n=== {role.upper()} ===")
        
        # Print content with proper wrapping
        wrapper = textwrap.TextWrapper(
            width=width,
            break_long_words=False,
            replace_whitespace=False
        )
        
        # Handle multi-line content
        for line in content.split('\n'):
            if line.strip():  # Skip wrapping empty lines
                wrapped_lines = wrapper.wrap(line)
                for wrapped in wrapped_lines:
                    print(wrapped)
            else:
                print()  # Empty line
        
        print("-" * width)

In [57]:
def extract_thought_solution(text):
    thought = ""
    solution = ""
    
    # Extract thought
    thought_start = text.find("<|begin_of_thought|>")
    thought_end = text.find("<|end_of_thought|>")
    if thought_start != -1 and thought_end != -1:
        thought = text[thought_start + len("<|begin_of_thought|>"):thought_end].strip()
    
    # Extract solution
    solution_start = text.find("<|begin_of_solution|>")
    solution_end = text.find("<|end_of_solution|>")
    if solution_start != -1 and solution_end != -1:
        solution = text[solution_start + len("<|begin_of_solution|>"):solution_end].strip()
    
    return thought, solution

In [81]:
prompt_template = """
You are provided with a data sample from a mathematical reasoning dataset. This sample includes a 'Question', a detailed 'Thought' process for solving the question, and the final 'Solution'.

Your task is to condense the 'Thought' section of the provided data sample.

**Goal:** Reduce the length of the 'Thought' text while preserving the essential reasoning steps, problem-solving logic, mathematical calculations, and any crucial verification steps that led to the solution. Remove verbose explanations, redundant phrases, or conversational filler, focusing on the core thought process.

**Constraint:** The condensed 'Thought' must be no more than {token_limit} tokens in length.

**Output Format:** Return *only* the condensed 'Thought' section. Ensure it retains the original formatting, starting with `<|begin_of_thought|>` and ending with `<|end_of_thought|>`. Do not include the 'Question' or 'Solution' sections in your output.

**Process the following data sample:**

=== DATA SAMPLE ===
=== QUESTION ===
{question_text}

=== THOUGHT ===
{thought_text}

=== SOLUTION ===
{solution_text}
=== END DATA SAMPLE ===

Please provide only the condensed 'Thought' section below, adhering to the token limit (no more than {token_limit}) and focusing on the core reasoning.
"""

In [92]:
example = ot.shuffle()[0]
thought, solution = extract_thought_solution(example["conversations"][1]["value"])
question = example["conversations"][0]["value"]

token_limit = len(thought.split()) // 200 * 100
prompt = prompt_template.format(
    token_limit=token_limit,
    question_text=question,
    thought_text=thought,
    solution_text=solution
)

In [93]:
print(prompt)


You are provided with a data sample from a mathematical reasoning dataset. This sample includes a 'Question', a detailed 'Thought' process for solving the question, and the final 'Solution'.

Your task is to condense the 'Thought' section of the provided data sample.

**Goal:** Reduce the length of the 'Thought' text while preserving the essential reasoning steps, problem-solving logic, mathematical calculations, and any crucial verification steps that led to the solution. Remove verbose explanations, redundant phrases, or conversational filler, focusing on the core thought process.

**Constraint:** The condensed 'Thought' must be no more than 1400 tokens in length.

**Output Format:** Return *only* the condensed 'Thought' section. Ensure it retains the original formatting, starting with `<|begin_of_thought|>` and ending with `<|end_of_thought|>`. Do not include the 'Question' or 'Solution' sections in your output.

**Process the following data sample:**

=== DATA SAMPLE ===
=== QUEST

In [95]:
x = """
<|begin_of_thought|>
Problem understanding: We have 3 doors (1, 2, 3) and 3 corresponding keys. One key x is in hand. Two keys are behind two different doors (the third door has no key, value 0). Keys 1, 2, 3 appear exactly once among x and the non-zero keys behind doors a, b, c (where a, b, c are keys behind doors 1, 2, 3 respectively). We can open a door if we have the key. Opening a door gives us the key behind it (if any). Can we open all three doors?

The process involves using the initial key x to open the corresponding door. If successful and a key is found, we add it to our collection and try opening the corresponding door. This continues until we can open all doors or cannot open any more doors.

Let's trace the examples:

Input: x=3, keys behind doors a=0, b=1, c=2. (Door 1: key 0, Door 2: key 1, Door 3: key 2).

Start with key 3.

Open door 3. Get key 2. Have keys {3, 2}.

Open door 2 (using key 2). Get key 1. Have keys {3, 2, 1}.

Open door 1 (using key 1). Get key 0 (nothing). Have keys {3, 2, 1}.

All doors (1, 2, 3) opened. Output: YES.

Input: x=1, keys behind doors a=0, b=3, c=2. (Door 1: key 0, Door 2: key 3, Door 3: key 2).

Start with key 1.

Open door 1. Get key 0. Have keys {1}.

Cannot open door 2 (need key 2) or door 3 (need key 3).

Stuck. Only door 1 opened. Output: NO.

This confirms the process: simulate the unlocking, collect keys, and see if all doors can eventually be opened.

Algorithm design:

Use a set current_keys to store keys currently held. Initialize with x.

Use a set opened_doors to track which doors (1, 2, 3) are opened. Initialize empty.

Use a mapping (e.g., a list or dictionary) to store which key is behind which door. door_keys = [a, b, c], where door_keys[0] is key behind door 1, door_keys[1] behind door 2, door_keys[2] behind door 3.

Loop repeatedly:

In each iteration, find all doors d (from 1, 2, 3) such that d is in current_keys AND d is not in opened_doors.

If no such doors are found, break the loop (no more progress possible).

For each found door d:

Add d to opened_doors.

Get the key k behind door d (which is door_keys[d-1]).

If k is not 0, add k to current_keys.

After the loop terminates, check if len(opened_doors) == 3. If yes, output YES, otherwise NO.

This iterative approach ensures we use newly acquired keys. The order of opening doors within an iteration doesn't matter since all collected keys become available for the next iteration.

Implementation details:

Map door numbers 1, 2, 3 to list indices 0, 1, 2 for door_keys. So, door d corresponds to index d-1.

The loop condition can be implemented using a flag or by comparing the size of opened_doors before and after an iteration.

Refined pseudo-code:

read t
for each testcase:
  read x
  read a, b, c
  door_keys = [a, b, c]
  current_keys = {x}
  opened_doors = set()

  while True:
    made_progress = False
    doors_to_open_this_round = []
    for door_num in [1, 2, 3]:
      if door_num not in opened_doors and door_num in current_keys:
        doors_to_open_this_round.append(door_num)

    if not doors_to_open_this_round:
      break # Cannot open any more doors

    for door_num in doors_to_open_this_round:
      if door_num not in opened_doors: # Check again in case multiple keys open same door? No, keys are unique. Redundant check likely.
        opened_doors.add(door_num)
        key_behind = door_keys[door_num - 1]
        if key_behind != 0:
          current_keys.add(key_behind)
        made_progress = True # Progress was made in this round

    # Optimization: If no progress was made in the inner loop over doors_to_open_this_round, we can break.
    # The outer check `if not doors_to_open_this_round: break` already handles this.

  if len(opened_doors) == 3:
    print("YES")
  else:
    print("NO")


The check made_progress isn't strictly necessary due to the if not doors_to_open_this_round: break check which happens before attempting to open.

Let's dry run the other examples with this logic:
3. Input: x=2, a=3, b=1, c=0. door_keys = [3, 1, 0].

current_keys = {2}, opened_doors = {}.

Iteration 1:

Check doors 1, 2, 3. Only door 2 is in current_keys. doors_to_open = [2].

Open door 2. opened_doors = {2}. Key behind is door_keys[1] = 1. Add 1 to current_keys. current_keys = {1, 2}.

Iteration 2:

Check doors 1, 3 (2 is open). Door 1 is in current_keys. Door 3 is not. doors_to_open = [1].

Open door 1. opened_doors = {1, 2}. Key behind is door_keys[0] = 3. Add 3. current_keys = {1, 2, 3}.

Iteration 3:

Check door 3 (1, 2 open). Door 3 is in current_keys. doors_to_open = [3].

Open door 3. opened_doors = {1, 2, 3}. Key behind is door_keys[2] = 0. Nothing to add. current_keys = {1, 2, 3}.

Iteration 4:

Check doors - none meet criteria (all open). doors_to_open = []. Break.

Result: len(opened_doors) == 3. Output: YES. Matches example.

Input: x=2, a=1, b=3, c=0. door_keys = [1, 3, 0].

current_keys = {2}, opened_doors = {}.

Iteration 1:

Check doors 1, 2, 3. Only door 2 is in current_keys. doors_to_open = [2].

Open door 2. opened_doors = {2}. Key behind is door_keys[1] = 3. Add 3. current_keys = {2, 3}.

Iteration 2:

Check doors 1, 3 (2 is open). Door 1 needs key 1 (not in {2, 3}). Door 3 needs key 3 (in {2, 3}). doors_to_open = [3].

Open door 3. opened_doors = {2, 3}. Key behind is door_keys[2] = 0. Nothing to add. current_keys = {2, 3}.

Iteration 3:

Check door 1 (2, 3 open). Door 1 needs key 1 (not in {2, 3}). doors_to_open = []. Break.

Result: len(opened_doors) == 2. Output: NO. Matches example.

Consider edge cases:

What if the key needed is behind the door itself? E.g., x=1, door 1 has key 1? This is impossible by problem constraints: "Values 1, 2 and 3 appear exactly once among x, a, b and c". So x cannot equal a, b, or c if they are non-zero.

What if the initial key x opens a door with key 0? Example: x=2, a=3, b=0, c=1. door_keys = [3, 0, 1].

current_keys = {2}. Open door 2. Get key 0. opened_doors = {2}, current_keys = {2}.

Cannot open door 1 (need 1) or door 3 (need 3). Loop ends. Output: NO. The algorithm handles this.

Cycles: E.g., x=1, door 1 has key 2, door 2 has key 3, door 3 has key 1.

current_keys = {1}. Open door 1, get 2. keys = {1, 2}.

Open door 2, get 3. keys = {1, 2, 3}.

Open door 3, get 1 (already have). All doors open. YES.

Unproductive cycles: E.g., x=1, door 1 has key 2, door 2 has key 1, door 3 has key 3.

current_keys = {1}. Open door 1, get 2. keys = {1, 2}.

Open door 2, get 1. keys = {1, 2}.

Cannot open door 3 (need 3). Loop ends. Output: NO. The algorithm handles this too.

The logic seems robust and covers the problem constraints and scenarios tested. The implementation using sets for keys and opened doors, and a list for keys behind doors, should work correctly. The time complexity per test case is small, essentially constant time since there are only 3 doors and the loop runs at most 3-4 iterations.
Final check on constraints: t <= 18, x in {1, 2, 3}, a, b, c in {0, 1, 2, 3}. Keys 1, 2, 3 appear exactly once among x, a, b, c. Everything fits.
The proposed algorithm correctly simulates the process and should yield the right answer.

# Python code sketch based on the logic:
def solve():
    x = int(input())
    a, b, c = map(int, input().split())
    door_keys = [a, b, c] # door_keys[0] for door 1, etc.

    current_keys = {x}
    opened_doors = set()

    while True:
        can_open_now = []
        for door_num in [1, 2, 3]:
            if door_num not in opened_doors and door_num in current_keys:
                can_open_now.append(door_num)

        if not can_open_now:
            break # No more doors can be opened with current keys

        for door in can_open_now:
            opened_doors.add(door)
            key_found = door_keys[door - 1]
            if key_found != 0:
                current_keys.add(key_found)

    if len(opened_doors) == 3:
        print("YES")
    else:
        print("NO")

# t = int(input())
# for _ in range(t):
#    solve()
IGNORE_WHEN_COPYING_START
content_copy
download
Use code with caution.
Python
IGNORE_WHEN_COPYING_END

This code structure matches the logic developed.
<|end_of_thought|>
""".strip()
len(x.split())

1391